#### Directories etc

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import random
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
import random
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
from Comparative_Analysis import ORF_Functions as orffn
from Comparative_Analysis import Alignment as alignfn
from random import sample
from Bio.Align.Applications import MuscleCommandline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
project_dir = 'F:/Project_Data/E_Pump_Project'
seq_dir = 'F:/Datasets/NCBI_Refseq_Mycobacteriaceae_All_Levels/ncbi_dataset/data'
tb_species = 'NC_000962.3' 
tb_annotation_dirname = 'GCF_000195955.2'
num_cores = 16
core_numbers = list(range(1, num_cores+1))
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'
full_build = False

#### Create file with all CDS for species and create BLAST database

In [47]:
sequence_dirs = util.list_dirs(seq_dir)
len(sequence_dirs)
len(all_cds)

1116061

In [4]:
loci = ['Rv0451c', 'Rv0450c', 'Rv0452']

In [7]:
def generate_protein_dataset(num_subsets, subset_num, dir_list): 
    sequence_dirs = util.chunk_list(dir_list, num_subsets, subset_num)
    tb_cds = []
    all_cds = []
    names = []
    sequences = []
    locations = []
    for dirname in (sequence_dirs):
        for genome_record in SeqIO.parse(seq_dir + '/' + dirname + '/genomic.gbff', "genbank"):
            accession_ver = genome_record.annotations['accessions'][0] + '.' + str(genome_record.annotations['sequence_version'])
            names.append([accession_ver, genome_record.annotations['organism']])
            full_sequence = str(genome_record.seq)
            sequences.append([accession_ver, full_sequence])
            for feature in genome_record.features:
                a = feature.qualifiers
                if feature.type == 'CDS' and a.get("translation") != None:
                    locus_tag = a.get("locus_tag")[0]
                    accession_locus = accession_ver + '@' + locus_tag
                    translation = a.get("translation")[0]
                    (start, stop, strand) = (int(feature.location.start), int(feature.location.end), int(feature.location.strand))
                    locations.append([accession_locus, (start, stop, strand)])
                    all_cds.append([accession_locus, translation])
                    if dirname == tb_annotation_dirname and locus_tag in loci:
                        tb_cds.append([accession_locus, translation])
    return (all_cds, tb_cds, names, locations, sequences)           

In [9]:
if full_build == True:
#if 1==1:
    parallel_output = Parallel(n_jobs=-1)(delayed(generate_protein_dataset)(num_cores, core_number, sequence_dirs) for core_number in core_numbers)
    names_dict = {}
    locations_dict = {}
    sequence_dict = {}
    all_cds = []
    tb_cds = []
    for x in parallel_output:
        all_cds += x[0]
        tb_cds += x[1]
        for temp in x[2]:
            names_dict[temp[0]] = temp[1]
        for temp in x[3]:
            locations_dict[temp[0]] = temp[1]
        for temp in x[4]:
            sequence_dict[temp[0]] = temp[1]

In [10]:
if full_build == True:
    util.produce_fasta_file(all_cds, project_dir + '/all_cds.fasta')
    util.produce_fasta_file(tb_cds, project_dir + '/tb_cds.fasta')
    blastfn.build_blast_db(project_dir, 'all_cds.fasta', 'Mycobacteriaceae_Refseq', 'F:/Datasets/BLAST/Mycobacteriaceae_Refseq')
    blastfn.build_blast_db(project_dir, 'tb_cds.fasta', 'E_Pump_CDS', 'F:/Datasets/BLAST/E_Pump_CDS')

In [11]:
if full_build == True:
    blastfn.run_blastp('F:/Datasets/BLAST/Mycobacteriaceae_Refseq', 'tb_cds.fasta', 'Mycobacteriaceae_Refseq', 'tb_cds_hits.csv', e_value = 1e-10)
    blastfn.run_blastp('F:/Datasets/BLAST/E_Pump_CDS', 'all_cds.fasta', 'E_Pump_CDS', 'reverse_hits.csv', e_value = 1e-10)

In [56]:
a = blastfn.process_blast_output('F:/Datasets/BLAST/Mycobacteriaceae_Refseq/tb_cds_hits.csv', names_dict, top_hit_only = False)
a = a.loc[a.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()]     # Utility top hit method uses accession ver which can differ if multiple sets exist per species in fragmented annotations
b = blastfn.process_blast_output('F:/Datasets/BLAST/E_Pump_CDS/reverse_hits.csv', names_dict, top_hit_only = False)
b = b.loc[b.groupby(['query_ref','target_species_name'])['bit_score'].idxmax()] 
rbh =  blastfn.keep_reciprocal_best_hits(a, b)

In [58]:
len(list(rbh['target_species_name'].unique()))

19

In [57]:
rbh['target_loc'] = rbh['target_ref'].map(locations_dict)
rbh['cds_count'] = rbh.groupby('target_species_name')['query_ref'].transform('size')
rbh['min_pct_id'] = rbh.groupby('target_species_name')['percent_identical_matches'].transform('min')
rbh = rbh[rbh['cds_count'] == 3]
for i, r in rbh.iterrows():
    (start, stop, strand) = r['target_loc']
    rbh.at[i,'start'] = start
    rbh.at[i,'stop'] =  stop
    rbh.at[i,'strand'] = strand
rbh['min_start'] = rbh.groupby('target_species_name')['start'].transform('min')
rbh['max_stop'] = rbh.groupby('target_species_name')['stop'].transform('max')
rbh['region_size'] = rbh['max_stop'] - rbh['min_start']
rbh = rbh[rbh['region_size'] < 5000]
rbh = rbh[rbh['min_pct_id'] >= 80]
rbh.to_csv(project_dir + '/reciprocal_best_hits.csv')

In [26]:
intergenic_regions = []
for target_species in list(rbh['target_species_name'].unique()):
    temp = rbh[rbh['target_species_name'] == target_species]
    for i, r in temp.iterrows():
        target_species_accession = r['target_species']
        if r['query_ref'].split('@')[1] == 'Rv0451c':
            Rv_0451c_coords = r['target_loc']
        if r['query_ref'].split('@')[1] == 'Rv0452':
            Rv_0452_coords = r['target_loc']
    if Rv_0451c_coords[2] == 1:
        intergenic_region = (Rv_0452_coords[1], Rv_0451c_coords[0]+3, 1)
    else:
        intergenic_region = (Rv_0451c_coords[1] - 3, Rv_0452_coords[0], -1)
    if intergenic_region[2] == 1:
        intergenic_sequence = sequence_dict[target_species_accession][intergenic_region[0]: intergenic_region[1]]
    else:
        intergenic_sequence = util.reverse_complement(sequence_dict[target_species_accession][intergenic_region[0]: intergenic_region[1]])
    print(target_species, intergenic_region)
    intergenic_regions.append([target_species.replace(" ", "_"), intergenic_sequence])

Mycobacterium attenuatum (48680, 49020, -1)
Mycobacterium basiliense (748594, 748891, -1)
Mycobacterium canettii CIPT 140010059 (552190, 552424, -1)
Mycobacterium decipiens (235054, 235252, 1)
Mycobacterium gastri (4687, 5006, 1)
Mycobacterium innocens (13755, 14137, -1)
Mycobacterium intermedium (13287, 13640, -1)
Mycobacterium kansasii ATCC 12478 (3978808, 3979155, -1)
Mycobacterium lacus (3937505, 3937835, 1)
Mycobacterium marinum (2347680, 2348049, -1)
Mycobacterium ostraviense (4846888, 4847225, 1)
Mycobacterium persicum (3848616, 3848956, -1)
Mycobacterium pseudokansasii (691242, 691597, -1)
Mycobacterium pseudoshottsii JCM 15466 (983982, 984351, -1)
Mycobacterium riyadhense (5455021, 5455277, 1)
Mycobacterium shinjukuense (3257603, 3257841, 1)
Mycobacterium shottsii (2404843, 2405212, -1)
Mycobacterium simulans (441647, 441928, 1)
Mycobacterium tuberculosis H37Rv (541907, 542141, -1)


In [27]:
util.produce_fasta_file(intergenic_regions, project_dir +'/intergenic_regions.fasta')
cline = MuscleCommandline(muscle_exe, input= project_dir +'/intergenic_regions.fasta', out=project_dir + '/intergenic_region_alignment.fasta')
result = cline();

100%|██████████| 19/19 [00:00<?, ?it/s]


In [17]:
blastfn.convert_fasta_to_stockholm(project_dir, 'intergenic_region_alignment.fasta', 'intergenic_region_alignment.sto')

In [18]:
blastfn.run_rscape(project_dir, 'intergenic_region_alignment.sto', 'rscape_output')

In [40]:
a = (sequence_dict[tb_species][542141:542852])

In [46]:
for i in range(len(a) - 3):
    if a[i:i+3] == 'TTT':
        print(a[i:i+10])


TTTTGTGATC
TTTGTGATCT
TTTGCTGCCG


In [45]:
a[1:4]

'TGC'

#### Set up dataframe with annotated (based on PGP run) M.tb feature boundaries and Mycobrowser M.tb feature boundaries

#### Produce FASTA file with CDS candidates

In [ ]:
util.produce_fasta_file(all_cds, project_dir + '/all_cds.fasta')
util.produce_fasta_file(tb_cds, project_dir + '/tb_cds.fasta')

In [ ]:
blastfn.build_blast_db(project_dir, 'all_cds.fasta', 'all_cds', 'F:/Datasets/BLAST/all_cds')
blastfn.build_blast_db(project_dir, 'tb_cds.fasta', 'tb_cds', 'F:/Datasets/BLAST/tb_cds')

In [ ]:
blastfn.run_blastp('F:/Datasets/BLAST/all_cds', 'tb_cds.fasta', 'all_cds', 'tb_cds_hits.csv', e_value = 1e-10)

#### Analyse CDS orthologs in target species

In [ ]:
blast_output = blastfn.process_blast_output('F:/Datasets/BLAST/all_cds/tb_cds_hits.csv', names_dict, top_hit_only = False)

In [ ]:
test = blast_output #blast_output[blast_output['target_species_name'] == 'Mycobacterium marinum'];
test['num_identical_matches'] = test['alignment_length'] * test['percent_identical_matches']/100;
test['max_identical_query_matches'] = test.groupby(['query_ref', 'target_species'])['num_identical_matches'].transform('max');
test['max_identical_target_matches'] = test.groupby('target_ref')['num_identical_matches'].transform('max');
test = test[test['num_identical_matches']>0.9999*test['max_identical_query_matches']]
test = test[test['num_identical_matches']>0.9999*test['max_identical_target_matches']]
test['count_matches'] = test.groupby('query_ref')['target_ref'].transform('count');
test['upstream_sequence'] = test['target_ref'].map(upstream_seq_dict)
test['annotated_translation'] = test['target_ref'].map(annotated_orf_translation_dict)
test['maximal_translation'] = test['target_ref'].map(maximal_orf_translation_dict)
test.to_csv(project_dir + '/test.csv')

In [ ]:
query_refs = [tb_species + '@' + locus for locus in loci]

In [ ]:
for ref in query_refs:
    print(ref)
    sequences = []
    temp = test[test['query_ref'] == ref]
    for i, r in temp.iterrows():
        sequences.append([r['target_ref'], r['upstream_sequence']])
    util.produce_fasta_file(sequences, project_dir + '/sequences_to_align.faa')
    cline = MuscleCommandline(muscle_exe, input= project_dir+'/'+ '/sequences_to_align.faa', out=project_dir + '/align_output_'+ref )
    result = cline();

In [ ]:
query_ref = 'NC_000962.3@Rv1410c'
align = alignfn.Alignment(project_dir + '/align_output_'+query_ref, query_ref, 'NT', insert_symbol = '-', species_order = [])

In [ ]:
align.calculate_entropies(modified=False)

In [ ]:
align.sequence_list

In [ ]:
sns.scatterplot(data = temp_df, x = 'query_1_start', y = 'target_1_start', s=1)

In [ ]:
test.to_csv(project_dir + '/test.csv')

In [ ]:
sns.scatterplot(data = test, x  = 'tb_start', y = 'target_start', s=1 )

In [ ]:
sns.histplot(data = test, x = 'percent_identical_matches')

#### Analyse interfeature orthologs in target species

In [ ]:
prob_dict = {}
for (start, stop, strand, length, prob) in mycobrowser_inter_feature_orf_probabilities:
    prob_dict[(start, stop, strand)] = prob

In [ ]:
if_blast_output = blastfn.process_blast_output('F:/Datasets/BLAST/comparator_orfs/if_hits.csv', organism_name_dict, top_hit_only = False)
if_test = if_blast_output[if_blast_output['target_species_name'] == 'Mycobacterium marinum'];
if_test['Overlap'] = 0
if_test['Genbank_Coordinates'] = ''

for i, r in tqdm(if_test.iterrows()):
    start = int(r['query_ref'].split('@')[1].split('_')[0])
    stop = int(r['query_ref'].split('@')[1].split('_')[1])
    strand = int(r['query_ref'].split('@')[1].split('_')[2])
    if_test.at[i,'probability'] = prob_dict[(start, stop, strand)]
    highest_overlap = 0
    hit = 0
    for feature in genbank_cds_boundaries:
            if start <= feature[1] and stop >= feature[0]:
                overlap = (min(stop, feature[1]) - max(start, feature[0]))/(stop - start)
                if overlap > highest_overlap:
                    hit = 1
                    highest_overlap = overlap
                    highest_entry = feature
    if hit == 1:
        if_test.at[i,'Overlap'] = highest_overlap
        if_test.at[i,'Genbank_Coordinates'] = highest_entry
if_test = if_test[abs(if_test['query_length'] - if_test['query_end_alignment']) < 3]
if_test = if_test[abs(if_test['subject_length'] - if_test['target_end_alignment']) < 3]
if_test.to_csv(project_dir + '/if_test_2.csv')

In [ ]:
of = orffn.H37Rv_ORF_Finder()
print(of.mutation_count_list[4056047:4056344])
print(of.mutation_bin_probability(of.mutation_count_list[4056047:4056344]))

In [ ]:
average = []
totlen = len(of.mutation_count_list)
for x in range(0, int(totlen/500)-1):
    average.append(sum(of.mutation_count_list[x*500: x*500 +500]))
sns.histplot(average, bins=300)

In [ ]:
def bin_formula(max_bin_counts, tot_bin_counts):
    return 1- binom.cdf(max_bin_counts-1, tot_bin_counts,1/3)

def mutation_bin_probability(mutation_counts):
    bin_counts = [0,0,0]
    for i, c in enumerate(mutation_counts):
        bin_counts[i % 3] += c
    if sum(bin_counts) == 0:
        return 2
    else:
        return (bin_counts, bin_formula(bin_counts[2], sum(bin_counts)))  

In [ ]:
mutation_bin_probability(of.mutation_count_list[4056047:4056344])

In [ ]:
organism_name_dict

In [ ]:
orf_dict['NC_000962.3'][0][2760753:2760856]

In [ ]:
orf_dict['NZ_CP058277.1'][0][5934538:5934641]

In [ ]:
orf_dict['NZ_AP022581.1'][0][692416:692519]

In [ ]:
trans = util.Translator()
v = orf_dict[tb_species]'NC_000962.3@2760763_2762380_1']
   

In [ ]:
 for k, v in orf_dict.items():
        full_sequence = v[0]
        orf_list = v[1]
        for x in orf_list:
            if x[2] == 1:
                prot = trans.translate_sequence(full_sequence[x[0]:x[1]], 1, 0)

In [ ]:
trans = util.Translator()
temp = []
for k, v in orf_dict.items():
    full_sequence = v[0]
    orf_list = v[1]
    for x in orf_list:
        if x[2] == 1:
            prot = trans.translate_sequence(full_sequence[x[0]:x[1]], 1, 0)
        else:
            prot = trans.translate_sequence(util.reverse_complement(full_sequence[x[0]:x[1]]), 1, 0)
        name = k + '@' +str(x[0])+'_'+str(x[1])+'_'+str(x[2])
        if name in ['NC_000962.3@2760763_2762380_1']:
            print(prot[:-1])
        if name in ['NZ_CP058277.1@5934638_5936567_1']:
            print(prot[89:622])

In [ ]:
cds_info_dict['NZ_CP080999.1']['K3U93_RS21900']